I went a slightly different approach here, originally I had written a function for a model and used this variable ```feature_cols``` to select the columns I was interested in. As I'm not dynamically changing these to demonstrate things as per the book, I did not include this and instead just dumped my bike data into X after making my dummy variables and removing the cound.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


def when_is_it(hour):
    if hour >=5 and hour < 11:
        return 'morning'
    elif hour >=11 and hour < 16:
        return 'afternoon'
    elif hour >=16 and hour < 18:
        return 'rush_hour'
    else:
        return 'off_hours'

def season_is_it(season_int):
    if season_int == 1:
        return 'spring'
    elif season_int == 2:
        return 'summer'
    elif season_int == 3:
        return 'fall'
    else:
        return 'winter'
    
def weather_is_it(weather_int):
    if weather_int == 1:
        return 'nice'
    elif weather_int == 2:
        return 'misty'
    elif weather_int == 3:
        return 'ugly'
    else:
        return 'stormy'
    
#function takes all nominal features and converts to dummy features
def dummy_conversion(data): 
    #create the rules for each dummy variable
    df_when_is_it = data['when_is_it'].apply(when_is_it)
    df_season_is_it = data['season'].apply(season_is_it)
    df_weather_is_it = data['weather'].apply(weather_is_it)
    
    
    when_dummies = pd.get_dummies(df_when_is_it, prefix = 'when_')    
    season_dummies = pd.get_dummies(df_season_is_it, prefix = 'season_')
    weather_dummies = pd.get_dummies(df_weather_is_it, prefix = 'weather_')
    
    #drop the old nominal veriables
    data=data.drop('datetime', axis = 1 )
    data=data.drop('season', axis = 1)
    data=data.drop('weather', axis = 1)
    
    data[list(when_dummies.columns)] = when_dummies
    data[list(season_dummies.columns)] = season_dummies
    data[list(weather_dummies.columns)] = weather_dummies
    return data                 

In [ ]:
from azureml.core import Workspace
we = Workspace.from_config()

In [2]:
    
bikes = pd.read_csv('train.csv')

test_data = pd.read_csv('test.csv')
temp_datetime = pd.read_csv('test.csv')['datetime']

feature_cols = ['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed']    

y = bikes['count']
bikes=bikes[feature_cols]

bikes['when_is_it']= bikes['datetime'].apply(lambda x:int(x[11]+x[12]))
test_data['when_is_it'] = test_data['datetime'].apply(lambda x:int(x[11]+x[12]))
                      



X=dummy_conversion(bikes)
test_data=dummy_conversion(test_data)

In [3]:
X.head()

,holiday,workingday,temp,atemp,humidity,windspeed,when_is_it,when__afternoon,when__morning,when__off_hours,when__rush_hour,season__fall,season__spring,season__summer,season__winter,weather__misty,weather__nice,weather__stormy,weather__ugly
0,0,0,9.84,14.395,81,0.0,0,0,0,1,0,0,1,0,0,0,1,0,0
1,0,0,9.02,13.635,80,0.0,1,0,0,1,0,0,1,0,0,0,1,0,0
2,0,0,9.02,13.635,80,0.0,2,0,0,1,0,0,1,0,0,0,1,0,0
3,0,0,9.84,14.395,75,0.0,3,0,0,1,0,0,1,0,0,0,1,0,0
4,0,0,9.84,14.395,75,0.0,4,0,0,1,0,0,1,0,0,0,1,0,0


In [4]:
X.shape

(10886, 19)

In [5]:
test_data.head()

,holiday,workingday,temp,atemp,humidity,windspeed,when_is_it,when__afternoon,when__morning,when__off_hours,when__rush_hour,season__fall,season__spring,season__summer,season__winter,weather__misty,weather__nice,weather__stormy,weather__ugly
0,0,1,10.66,11.365,56,26.0027,0,0,0,1,0,0,1,0,0,0,1,0,0
1,0,1,10.66,13.635,56,0.0000,1,0,0,1,0,0,1,0,0,0,1,0,0
2,0,1,10.66,13.635,56,0.0000,2,0,0,1,0,0,1,0,0,0,1,0,0
3,0,1,10.66,12.880,56,11.0014,3,0,0,1,0,0,1,0,0,0,1,0,0
4,0,1,10.66,12.880,56,11.0014,4,0,0,1,0,0,1,0,0,0,1,0,0


In [6]:
bikes.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,when_is_it
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,4


In [7]:
test_data.shape

(6493, 19)

In [11]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y)
    #X_train[feature_cols] = scaler.transform(X_train[feature_cols])
    #X_test[feature_cols] = scaler.transform(X_test[feature_cols])
    #X_train and y_train will be used to train the model
    #X_test and y_test will be used to test the model
    #remember that all four of these variables are just subsets of the overall X and Y

linreg = LinearRegression()
    #instantiate the model

linreg.fit(X_train, y_train)
    # fit the model to our training set

y_pred = linreg.predict(X_test)
    #predict our testing set
    #y_test = y_test.abs()
    
    #the log error doesn't take negative values, linear regression however can predict negative counts.
    #since this makes no sense, I make any negative predictions = 0.
y_pred[y_pred <0] = 0
    
error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
    #error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
    # calculate our metric
    
print('RMSLE is ' + str(error))
    



RMSLE is 1.1271433651887373


That's a nice improvement (I forgot to write it down before scaling! The RMSLE before doing the dummy variables was 1.37 - and that wasn't the average performance, that was after I ran it a number of times to get a better score LOL 

UPDATE: I wrote this comment after running this without even doing ANY scaling - I accidentally forgot to put the scaler in, I assume things will get even better with scaling

UPDATE2: After scaling it seems to have made things even better, we seem to kind of get 1.17 or 1.18 oftentimes, which is a lot of improvement! Best I could get after running a bunch was 1.13 and I am going with this for my submission! :)

In [12]:
scaler.fit(test_data)
test_data=scaler.transform(test_data)

predictions = linreg.predict(test_data)
#ensure all negative values predict 0
predictions[predictions <0] = 0 

#ensure all counts are rounded
predictions = np.rint(predictions)
submission = pd.DataFrame({'datetime': temp_datetime, 'count': predictions})

In [13]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,0.0
1,2011-01-20 01:00:00,0.0
2,2011-01-20 02:00:00,0.0
3,2011-01-20 03:00:00,0.0
4,2011-01-20 04:00:00,6.0


In [14]:
submission.to_csv('linReg_submission2.csv', index=False, )

https://www.kaggle.com/c/bike-sharing-demand
Above is the link I submit to. I officially got 1.46 RMSLE on the site when I submitted without dummy variables, but included scaling. 

My new submission... GOT 1.199984 = 1.2 RMSLE WOOOOOOOOO